In [3]:
from sklearn.datasets import load_digits
digits=load_digits()
dir(digits)

['DESCR', 'data', 'images', 'target', 'target_names']

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(digits.data,digits.target,test_size=0.2)


In [9]:
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)
model.score(x_test, y_test)

0.9805555555555555

In [12]:
from sklearn.model_selection import cross_val_score

In [15]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),digits.data, digits.target, cv=5)

array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [16]:
cross_val_score(SVC(kernel='linear',C=5,gamma='auto'),digits.data, digits.target, cv=5)

array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [18]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),digits.data, digits.target, cv=5)

array([0.45277778, 0.46944444, 0.47910864, 0.47910864, 0.50139276])

In [19]:
import numpy as np

In [32]:
kernel=['linear','rbf']
c=[1,10,100]
Avg_scores={}
for k_val in kernel:
  for c_val in c:
    score=cross_val_score(SVC(kernel=k_val,C=c_val,gamma='auto'),digits.data, digits.target, cv=5)
    Avg_scores[k_val + '_' + str(c_val)]=np.average(score)

Avg_scores


{'linear_1': 0.9476973073351903,
 'linear_10': 0.9476973073351903,
 'linear_100': 0.9476973073351903,
 'rbf_1': 0.448545341999381,
 'rbf_10': 0.47636645001547506,
 'rbf_100': 0.47636645001547506}

In [33]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(digits.data, digits.target)
clf.cv_results_

{'mean_fit_time': array([0.48791523, 0.03959498, 0.4860044 , 0.03872552, 0.48318281,
        0.03886809]),
 'mean_score_time': array([0.06311774, 0.01164079, 0.06243367, 0.01140995, 0.06175957,
        0.01141052]),
 'mean_test_score': array([0.44854534, 0.94769731, 0.47636645, 0.94769731, 0.47636645,
        0.94769731]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'rank_test_score': array([6, 1, 4, 1, 4, 1], dtype=int32),
 'split0_test_score': array([0.41111111, 0.96

In [35]:
import pandas as pd
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.487915,0.005441,0.063118,0.002181,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.411111,0.450000,0.454039,0.448468,0.479109,0.448545,0.021761,6
1,0.039595,0.002112,0.011641,0.000411,1,linear,"{'C': 1, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
2,0.486004,0.005698,0.062434,0.001205,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
3,0.038726,0.001431,0.011410,0.000295,10,linear,"{'C': 10, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
4,0.483183,0.002557,0.061760,0.000895,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
5,0.038868,0.001486,0.011411,0.000292,20,linear,"{'C': 20, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1


In [36]:
df[['param_C','param_kernel','mean_test_score']]


,param_C,param_kernel,mean_test_score
0,1,rbf,0.448545
1,1,linear,0.947697
2,10,rbf,0.476366
3,10,linear,0.947697
4,20,rbf,0.476366
5,20,linear,0.947697


In [37]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [38]:
clf.best_score_

0.9476973073351903

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [61]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'GaussianNB': {
        'model':GaussianNB(),
        'params' : {
            
        }  
    },
    'MultinomialNB': {
        'model':MultinomialNB(),
        'params' : {
            
        } 
    },
    'DecisionTreeClassifier': {
        'model':DecisionTreeClassifier(),
        'params' : {'criterion':['gini']}
            
        }      
}

In [62]:
score = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(digits.data, digits.target)
    score.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df1 = pd.DataFrame(score,columns=['model','best_score','best_params'])
df1

,model,best_score,best_params
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.893737,{'n_estimators': 10}
2,logistic_regression,0.922114,{'C': 1}
3,GaussianNB,0.806928,{}
4,MultinomialNB,0.870350,{}
5,DecisionTreeClassifier,0.790809,{'criterion': 'gini'}
